In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from pandas.plotting import scatter_matrix

ioTInTData = pd.read_csv("data/IoTID20/IoT_Network_Intrusion_Dataset.csv" , low_memory=False)
ioTInTData = ioTInTData[ioTInTData['Cat'] != 'Mirai']
ioTInTData = ioTInTData.iloc[:,:80]
print('ioTInTData', ioTInTData.shape)

#ioTInTData = ioTInTData.fillna(np.nan, inplace=True)
ioTInTData.replace([np.inf, -np.inf], np.nan, inplace=True)
ioTInTData.dropna(inplace=True)
ioTInTData.fillna(0)
ioTInTData.shape

def readDataSets():
    dataSetFridge = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_Fridge.csv')
    dataSetGarageDoor = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_Garage_Door.csv')
    dataSetGPS = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_GPS_Tracker.csv')
    dataSetModbus = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_Modbus.csv')
    dataSetMotionLight = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_Motion_Light.csv')
    dataSetThermostat = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_Thermostat.csv')
    dataSetWeahter = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/binary/IoT_Weather.csv')

    dataSetFridge['temp_condition'] = dataSetFridge['temp_condition'].str.strip()
    dataSetGarageDoor['door_state'] = dataSetGarageDoor['door_state'].str.strip()
    dataSetMotionLight['light_status'] = dataSetMotionLight['light_status'].str.strip()
    dataSetRawLoad = pd.concat([dataSetFridge, dataSetGarageDoor, dataSetGPS, dataSetModbus, dataSetMotionLight, dataSetThermostat, dataSetWeahter])
    return dataSetRawLoad

In [ ]:
df = readDataSets()
print(df.shape)

In [ ]:
# Load the data
categorical_features = ['door_state','sphone_signal', 'light_status','temp_condition']
quantitative_features = ['FC1_Read_Input_Register','FC2_Read_Discrete_Value','FC3_Read_Holding_Register','FC4_Read_Coil','current_temperature',
                        'fridge_temperature','humidity','latitude','FC4_Read_Coil','longitude',
                        'motion_status','pressure','temperature','thermostat_status']
features = categorical_features + quantitative_features


def datapreprocessingShuffle(data):
    scaler = StandardScaler()            
    # Feature scaling
    for i in data.columns:
        if(data[i].name !='Label'):
            data[i] = scaler.fit_transform(data[[i]])
        else:
            data[i]=data[i]
            
    data = shuffle(data).reset_index(drop=True) 
    return data


def datapreprocessingShuffle2(data):
               
    # Feature scaling
    for i in quantitative_features :
            scaler = StandardScaler()
            data[i] = scaler.fit_transform(data[[i]])
            
    # Encoding categorical features    
    for i in categorical_features :
        labelencoder=LabelEncoder()
        data[i]=labelencoder.fit_transform(data[i])   
    
    data = shuffle(data).reset_index(drop=True) 
    return data

In [ ]:
# Pre-processing datset
datacopy1 = ioTInTData.copy()
datacopy2 = df.copy()
datacopy1['Label'] = datacopy1.iloc[:,79:80].replace('Normal', '0').replace('Anomaly', '1').astype('int')
data1 = datapreprocessingShuffle(datacopy1)

data2 = datacopy2.rename(columns={"label": "Label"})
data2 = datapreprocessingShuffle2(data2) 
# fill na with mean for ToN_IoT dataset
data2 = data2.fillna(data2.mean())
print(data1.shape)
print(data2.shape)

In [ ]:
# get a list of columns
cols = list(data2)
# move the column to head of list using index, pop and insert
cols.insert(18, cols.pop(cols.index('Label')))

# use ix to reorder
data2 = data2.ix[:, cols]

In [ ]:
# for false labels
#data2 = data2.loc[data2['Label'] == 0] #no attack

# for false labels
data2 = data2.loc[data2['Label'] == 1] #attack

In [ ]:
import GAN_171103
import importlib
importlib.reload(GAN_171103) # For reloading after making changes
from GAN_171103 import *


 # 32 # needs to be ~data_dim
base_n_count = 128 # 128

nb_steps = 500 + 1 # 50000 # Add one for logging of the last interval
batch_size = 128 # 64

k_d = 1  # number of critic network updates per adversarial training step
k_g = 1  # number of generator network updates per adversarial training step
critic_pre_train_steps = 100 # 100  # number of steps to pre-train the critic before starting adversarial training
log_interval = 100 # 100  # interval (in steps) at which to log loss summaries and save plots of image samples to disc
learning_rate = 5e-4 # 5e-5
data_dir = 'cache/'
generator_model_path, discriminator_model_path, loss_pickle_path = None, None, None

# show = False
show = True 

# train = create_toy_spiral_df(1000)
# train = create_toy_df(n=1000,n_dim=2,n_classes=4,seed=0)
train = data2.copy().reset_index(drop=True) # fraud only with labels from classification


# train = pd.get_dummies(train, columns=['Class'], prefix='Class', drop_first=True)
label_cols = [ i for i in train.columns if 'Label' in i ]
data_cols = [ i for i in train.columns if i not in label_cols ]
train[ data_cols ] = train[ data_cols ] / 10 # scale to random noise size, one less thing to learn
train_no_label = train[ data_cols ]


rand_dim = len(data_cols)

In [103]:
%%time

# Training the vanilla GAN and CGAN architectures

k_d = 1  # number of critic network updates per adversarial training step
learning_rate = 5e-4 # 5e-5
arguments = [rand_dim, nb_steps, batch_size, 
             k_d, k_g, critic_pre_train_steps, log_interval, learning_rate, base_n_count,
            data_dir, generator_model_path, discriminator_model_path, loss_pickle_path, show ]

adversarial_training_GAN(arguments, train_no_label, data_cols ) # GAN
#adversarial_training_GAN(arguments, train, data_cols=data_cols, label_cols=label_cols ) # CGAN

In [ ]:
seed = 17

train = data2.copy().reset_index(drop=True) # fraud only with labels from classification

# train = pd.get_dummies(train, columns=['Class'], prefix='Class', drop_first=True)
label_cols = [ train.columns[-1]  ]
data_cols = [ i for i in train.columns if i not in label_cols ]
#train[ data_cols ] = train[ data_cols ] / 10 # scale to random noise size, one less thing to learn
train_no_label = train[ data_cols ]

data_dim = len(data_cols)
label_dim = len(label_cols)
#if label_dim > 0: with_class = True
np.random.seed(seed)

# define network models

generator_model, discriminator_model, combined_model = define_models_GAN(rand_dim, data_dim, base_n_count)
generator_model.load_weights('cache/GAN_generator_model_weights_step_500.h5')

# Now generate some new data
test_size = len(train) # Equal to all of the fraud cases

x = get_data_batch(train_no_label, test_size, seed=3)
z = np.random.normal(size=(test_size, rand_dim))
g_z = generator_model.predict(z)
    

# =============================================================================
# check data
# =============================================================================
df2=pd.DataFrame(np.rint(np.abs(g_z)),columns=df.columns[:-1])
df2=df2.astype(int)


def define_models_GAN(rand_dim, data_dim, base_n_count, type=None):
    generator_input_tensor = layers.Input(shape=(rand_dim, ))
    generated_image_tensor = generator_network(generator_input_tensor, data_dim, base_n_count)

    generated_or_real_image_tensor = layers.Input(shape=(data_dim,))
    
    if type == 'Wasserstein':
        discriminator_output = critic_network(generated_or_real_image_tensor, data_dim, base_n_count)
    else:
        discriminator_output = discriminator_network(generated_or_real_image_tensor, data_dim, base_n_count)

    generator_model = models.Model(inputs=[generator_input_tensor], outputs=[generated_image_tensor], name='generator')
    discriminator_model = models.Model(inputs=[generated_or_real_image_tensor],
                                       outputs=[discriminator_output],
                                       name='discriminator')

    combined_output = discriminator_model(generator_model(generator_input_tensor))
    combined_model = models.Model(inputs=[generator_input_tensor], outputs=[combined_output], name='combined')
    
    return generator_model, discriminator_model, combined_model

def generate_data_gan(n_samples,Xdf,path_model,rand_dim=32):
    generator_model, discriminator_model, combined_model= define_models_GAN(rand_dim,data_dim=len(Xdf.columns),base_n_count=128)
    generator_model.load_weights(path_model)
    z = np.random.normal(size=(n_samples, rand_dim))
    g_z = generator_model.predict(z)
    
    newdf = pd.DataFrame()
    for i, col in enumerate(Xdf.columns):

        if Xdf[col].dtype == 'int32' or Xdf[col].dtype == 'int64':
            newdf[col] = np.rint(np.abs(g_z[:,i])).astype(int)    
        elif Xdf[col].dtype == 'float_':
            newdf[col] = np.abs(g_z[:,i]) 
    return newdf


#rand_dim = 32 
#base_n_count = 128
#data_dim = len(data_cols) 
#'nursery','mushroom'

#data11 = 119698

#data12-FALSE = 62% of data1 (0.62 * 119698)= 74213 Label = 0
#data12-TRUE  = 38% of data1 (0.38 * 119698)= 45485 Label = 1
# data2 False



n_samples=45485

# data2 True
#n_samples=45485

Xdf=data2.iloc[:,:-1]    
path_model='cache/GAN_generator_model_weights_step_500.h5'

#data12False=generate_data_gan(n_samples,Xdf,path_model,rand_dim=len(data_cols))  
#data12False.shape

data12True=generate_data_gan(n_samples,Xdf,path_model,rand_dim=len(data_cols))  
data12True.shape

In [ ]:
data12False['Label'] = 0
print(data12False.shape)
data12True['Label'] = 1
print(data12True.shape)

In [ ]:
Gen_data2 = pd.concat([data12True, data12False])
Gen_data2.shape

# Data 1 Training

In [141]:
# for false labels
#data2 = data1.loc[data1['Label'] == 0] #no attack

# for false labels
data2 = data1.loc[data1['Label'] == 1] #attack

import GAN_171103
import importlib
importlib.reload(GAN_171103) # For reloading after making changes
from GAN_171103 import *


 # 32 # needs to be ~data_dim
base_n_count = 128 # 128

nb_steps = 500 + 1 # 50000 # Add one for logging of the last interval
batch_size = 128 # 64

k_d = 1  # number of critic network updates per adversarial training step
k_g = 1  # number of generator network updates per adversarial training step
critic_pre_train_steps = 100 # 100  # number of steps to pre-train the critic before starting adversarial training
log_interval = 100 # 100  # interval (in steps) at which to log loss summaries and save plots of image samples to disc
learning_rate = 5e-4 # 5e-5
data_dir = 'cache/'
generator_model_path, discriminator_model_path, loss_pickle_path = None, None, None

# show = False
show = True 

# train = create_toy_spiral_df(1000)
# train = create_toy_df(n=1000,n_dim=2,n_classes=4,seed=0)
train = data2.copy().reset_index(drop=True) # fraud only with labels from classification


# train = pd.get_dummies(train, columns=['Class'], prefix='Class', drop_first=True)
label_cols = [ i for i in train.columns if 'Label' in i ]
data_cols = [ i for i in train.columns if i not in label_cols ]
train[ data_cols ] = train[ data_cols ] / 10 # scale to random noise size, one less thing to learn
train_no_label = train[ data_cols ]


rand_dim = len(data_cols)

In [ ]:
%%time

# Training the vanilla GAN and CGAN architectures

k_d = 1  # number of critic network updates per adversarial training step
learning_rate = 5e-4 # 5e-5
arguments = [rand_dim, nb_steps, batch_size, 
             k_d, k_g, critic_pre_train_steps, log_interval, learning_rate, base_n_count,
            data_dir, generator_model_path, discriminator_model_path, loss_pickle_path, show ]

adversarial_training_GAN(arguments, train_no_label, data_cols ) # GAN
#adversarial_training_GAN(arguments, train, data_cols=data_cols, label_cols=label_cols ) # CGAN

In [ ]:
seed = 17

train = data2.copy().reset_index(drop=True) # fraud only with labels from classification

# train = pd.get_dummies(train, columns=['Class'], prefix='Class', drop_first=True)
label_cols = [ train.columns[-1]  ]
data_cols = [ i for i in train.columns if i not in label_cols ]
#train[ data_cols ] = train[ data_cols ] / 10 # scale to random noise size, one less thing to learn
train_no_label = train[ data_cols ]

data_dim = len(data_cols)
label_dim = len(label_cols)
#if label_dim > 0: with_class = True
np.random.seed(seed)

# define network models

generator_model, discriminator_model, combined_model = define_models_GAN(rand_dim, data_dim, base_n_count)
generator_model.load_weights('cache/GAN_generator_model_weights_step_500.h5')


# Now generate some new data
test_size = len(train) # Equal to all of the fraud cases

x = get_data_batch(train_no_label, test_size, seed=3)
z = np.random.normal(size=(test_size, rand_dim))
g_z = generator_model.predict(z)

 

# =============================================================================
# check data
# =============================================================================
df2=pd.DataFrame(np.rint(np.abs(g_z)),columns=data2.columns[:-1])
df2=df2.astype(int)



def define_models_GAN(rand_dim, data_dim, base_n_count, type=None):
    generator_input_tensor = layers.Input(shape=(rand_dim, ))
    generated_image_tensor = generator_network(generator_input_tensor, data_dim, base_n_count)

    generated_or_real_image_tensor = layers.Input(shape=(data_dim,))
    
    if type == 'Wasserstein':
        discriminator_output = critic_network(generated_or_real_image_tensor, data_dim, base_n_count)
    else:
        discriminator_output = discriminator_network(generated_or_real_image_tensor, data_dim, base_n_count)

    generator_model = models.Model(inputs=[generator_input_tensor], outputs=[generated_image_tensor], name='generator')
    discriminator_model = models.Model(inputs=[generated_or_real_image_tensor],
                                       outputs=[discriminator_output],
                                       name='discriminator')

    combined_output = discriminator_model(generator_model(generator_input_tensor))
    combined_model = models.Model(inputs=[generator_input_tensor], outputs=[combined_output], name='combined')
    
    return generator_model, discriminator_model, combined_model

def generate_data_gan(n_samples,Xdf,path_model,rand_dim=32):
    generator_model, discriminator_model, combined_model= define_models_GAN(rand_dim,data_dim=len(Xdf.columns),base_n_count=128)
    generator_model.load_weights(path_model)
    z = np.random.normal(size=(n_samples, rand_dim))
    g_z = generator_model.predict(z)
    
    newdf = pd.DataFrame()
    for i, col in enumerate(Xdf.columns):

        if Xdf[col].dtype == 'int32' or Xdf[col].dtype == 'int64':
            newdf[col] = np.rint(np.abs(g_z[:,i])).astype(int)    
        elif Xdf[col].dtype == 'float_':
            newdf[col] = np.abs(g_z[:,i]) 
    return newdf



#data22 = 401119

#data21-FALSE = 23% of data1 (0.23 * 401119)= 92257 Label = 0
#data21-TRUE  = 77% of data1 (0.77 * 401119)= 308862 Label = 1

n_samples=308862

# data2 True
#n_samples=45485

Xdf=data2.iloc[:,:-1]    
path_model='cache/GAN_generator_model_weights_step_500.h5'

#data21False=generate_data_gan(n_samples,Xdf,path_model,rand_dim=len(data_cols))  
#data21False.shape

data21True=generate_data_gan(n_samples,Xdf,path_model,rand_dim=len(data_cols))  
data21True.shape

In [ ]:
data22False['Label'] = 0
print(data22False.shape)
data22True['Label'] = 1
print(data22True.shape)

In [ ]:
Gen_data1 = pd.concat([data22True, data22False])
Gen_data1.shape

In [ ]:
# concat data22 with generated data12

#              Col1           |     Col2
#  Row1    data11 (IoTID20)  |  data12 (GAN)
#         _____________________________________
#          
#  Row2    data21 (GAN)     |  data22 (TonIoT)
#
# data22 = TonIoT
# data12 = Gen_data2
# Row1(520817) = data11(119698) + data12(401119)
Row1Data = pd.concat([data1, Gen_data2])
Row1Data.shape

In [ ]:
# concat data22 with generated data12

#    Col1           |     Col2
# data11 (IoTID20)  |  data12 (GAN)
# _____________________________________
#          
# data21 (GAN)     |  data22 (TonIoT)
#
# data11 = IoTID20
# data21 = Gen_data1
# Col1(520817) = data11(119698) + data21(401119)
row2Data = pd.concat([Gen_data1, data2])
row2Data.shape

In [ ]:
tData1 = data1.loc[data1['Label'] == 1] 
tData1

In [ ]:
totalData = pd.concat([col1Data, col2Data])
totalData.shape

In [ ]:
# fill na with mean for ToN_IoT dataset
fData = pd.concat([data1, data2])
fData = fData.fillna(fData.mean())
fData.shape

In [ ]:
pd.DataFrame(fData['Label'].value_counts())[:]

In [ ]:

#matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
carrier_count = fData['Label'].value_counts()
sns.set(style="darkgrid")
sns.barplot(carrier_count.index, carrier_count.values, alpha=0.9)
plt.title('Frequency Distribution of Cat')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Cat', fontsize=12)
plt.show()

In [ ]:
y = fData['Label']
X = fData.drop(['Label'],axis=1) 

print(y.shape)
print(X.shape)

from sklearn.preprocessing import MinMaxScaler
# on this distribution. 
sc = MinMaxScaler()
X_std =  sc.fit_transform(X)

cov_matrix = np.cov(X_std.T)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
# Make a set of (eigenvalue, eigenvector) pairs:
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]
# Sort the (eigenvalue, eigenvector) pairs from highest to lowest with respect to eigenvalue
#eig_pairs.sort()
eig_pairs.reverse()
#print(eig_pairs)
# Extract the descending ordered eigenvalues and eigenvectors
eigvalues_sorted = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sorted = [eig_pairs[index][1] for index in range(len(eigenvalues))]
# Let's confirm our sorting worked, print out eigenvalues
#print('Eigenvalues in descending order: \n%s' %eigvalues_sorted)

tot = sum(eigenvalues)
var_explained = [(i / tot) for i in sorted(eigenvalues, reverse=True)]  # an array of variance explained by each 
# eigen vector... there will be 18 entries as there are 18 eigen vectors)
cum_var_exp = np.cumsum(var_explained)  # an array of cumulative variance. There will be 18 entries with 18 th entry 
# cumulative reaching almost 100%

plt.bar(range(1,97), var_explained, alpha=0.5, align='center', label='individual explained variance')
plt.step(range(1,97),cum_var_exp, where= 'mid', label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc = 'best')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA

# feature selection
def select_featuresMutual(X, y):
    fs = SelectKBest(score_func=mutual_info_classif, k=4)
    X_new = fs.fit_transform(X, y)
    return X_new

# feature selection
def select_featuresPCA(X, y, n_comp):
    fs = PCA(n_components=n_comp)
    X_new = fs.fit_transform(X, y)
    return X_new

In [ ]:
# get a voting ensemble of models
def get_voting():
    # define the base models
    models = list()
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeRegressor()))
    models.append(('NB', GaussianNB()))
    models.append(('MLP', MLPClassifier()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('LR', LogisticRegression()))
    models.append(('SVM', svm.SVC()))
    models.append(('AdaBoost', AdaBoostClassifier()))
    models.append(('GradientBoosting', GradientBoostingClassifier()))
    models.append(('XGB', XGBClassifier()))
    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='soft')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = list()
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeRegressor()))
    models.append(('NB', GaussianNB()))
    models.append(('MLP', MLPClassifier()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('LR', LogisticRegression()))
    models.append(('SVM', svm.SVC()))
    models.append(('AdaBoost', AdaBoostClassifier()))
    models.append(('GradientBoosting', GradientBoostingClassifier()))
    models.append(('XGB', XGBClassifier()))
    models.append(('soft_voting', get_voting()))
    return models

def evaluate_model(model, X, y):
    cv = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models:
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [144]:
# evaluation of a model fit using mutual information input features
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# Spot Check Algorithms
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('MLP', MLPClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('LR', LogisticRegression()))
models.append(('SVM', svm.SVC()))
models.append(('AdaBoost', AdaBoostClassifier()))
models.append(('GradientBoosting', GradientBoostingClassifier()))
models.append(('XGB', XGBClassifier()))


for name, model in models:
    print('asd')
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    X_s = select_featuresMutual(X, y)
    cv_results = cross_val_score(model, X_s, y, cv=kfold, scoring='accuracy')
    print('%s: %s: %f (%f)' % ('Mutual-info', name, cv_results.mean(), cv_results.std()))
    X_s = select_featuresPCA(X, y, 25)
    cv_results = cross_val_score(model, X_s, y, cv=kfold, scoring='accuracy')
    print('%s: %s: %f (%f)' % ('PCA', name, cv_results.mean(), cv_results.std()))